<h1>Install the dependecies and create directory for the files</h1>

In [1]:
# %pip install llama-hub unstructured
# %pip install "unstructured[doc,docx,ppt,pptx,pdf,image]"
# %pip install llama-index
# %pip install llama-index
# %pip install unstructured
# curl -L https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh -o Miniforge3-MacOSX-arm64.sh
# bash Miniforge3-MacOSX-arm64.sh
# %pip install llama-cpp-python
# %pip install langchain
# %mkdir data

<h1>Load the data and construct the index</h1>

In [2]:
from pathlib import Path
import os
from llama_index import download_loader, SimpleDirectoryReader, VectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext, StorageContext, load_index_from_storage
# from langchain.chat_models import ChatOpenAI
from llama_index.llms import LlamaCPP
# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_hub.file.unstructured.base import UnstructuredReader
# from IPython.display import Markdown, display

UnstructuredReader = download_loader("UnstructuredReader")

def construct_index(directory_path):
    # # set maximum input size
    # max_input_size = 4096
    # # set number of output tokens
    # num_outputs = 2000
    # # set maximum chunk overlap
    # max_chunk_overlap = 0.2
    # # set chunk size limit
    # chunk_size_limit = 600

    # # define prompt helper
    # prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # # define LLM
    # llm_predictor = LLMPredictor(llm=ChatOpenAI(openai_api_key=openai.api_key, temperature=0.2, model_name="gpt-4", max_tokens=num_outputs))

    llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url=model_url,
    model_path="/Users/balungelir21/Documents/LlamaIndex_Chat/codellama-13b-instruct.Q5_K_M.gguf", #Add the model Path here!!!!!!!!!!!!
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
    )
    
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

    # check if storage already exists
    if(not os.path.exists("./storage")):
        # load the documents and create the index
        dir_reader = SimpleDirectoryReader(directory_path, file_extractor={
          ".txt": UnstructuredReader(),
          ".docx": UnstructuredReader(),
          ".pptx": UnstructuredReader(),
          ".pdf": UnstructuredReader(),
          ".jpg": UnstructuredReader(),
          ".png": UnstructuredReader(),
          ".eml": UnstructuredReader(),
          ".html": UnstructuredReader(),
        })
        documents = dir_reader.load_data()

        service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
        index = VectorStoreIndex.from_documents(documents, service_context=service_context)

        index.storage_context.persist()
    else:
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir="./storage")
        index = load_index_from_storage(storage_context)

    return index

<h1>Query the index</h1>

In [3]:
# query the index
def ask_jarvis():
    index = construct_index("./data")
    while True:
        query_engine = index.as_query_engine()
        query = input("Hello! What would you like to know?")
        response = query_engine.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

        

In [4]:
# construct_index("data")

In [5]:
ask_jarvis()

llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /Users/balungelir21/Documents/LlamaIndex_Chat/codellama-13b-instruct.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32016,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.we

APIConnectionError: Connection error.